In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import cv2

In [9]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)
        )
        self.fc = nn.Linear(32, num_classes)

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

# -------- Dataset Loader (CIFAR10 as example) --------
transform_raw = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_raw)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_raw)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)


In [24]:
from torchvision.transforms import ToPILImage, ToTensor
import cv2
import numpy as np
import torch

# Create instances of the transform functions
to_pil = ToPILImage()
to_tensor = ToTensor()

def apply_gaussian_blur(img):
    img_pil = to_pil(img)  # Convert tensor to PIL
    img_np = np.array(img_pil)  # Convert PIL to NumPy
    img_blurred = cv2.GaussianBlur(img_np, (5, 5), sigmaX=0)
    img_blurred_tensor = to_tensor(img_blurred)  # Back to tensor
    return img_blurred_tensor

In [32]:
# -------- Training loop --------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(num_classes=10).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

for epoch in range(75):  # Train only one epoch for testing
    for images, labels in train_loader:
        # Apply the non-differentiable filter to each image
        filtered = torch.stack([apply_gaussian_blur(img) for img in images])
        filtered = filtered.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(filtered)
        loss = criterion(outputs, labels)

        # Backward + Optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/75] - Loss: {loss / len(train_loader):.4f}")

# -------- Evaluation --------
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

acc = correct / total
print(f"Test Accuracy: {acc * 100:.2f}%")

# -------- Save Model --------
torch.save(model.state_dict(), "gaussian_blur_pipeline.pth")
print("Model saved to gaussian_blur_pipeline.pth")

Epoch [1/75] - Loss: 0.0012
Epoch [2/75] - Loss: 0.0013
Epoch [3/75] - Loss: 0.0011
Epoch [4/75] - Loss: 0.0012
Epoch [5/75] - Loss: 0.0009
Epoch [6/75] - Loss: 0.0012
Epoch [7/75] - Loss: 0.0012
Epoch [8/75] - Loss: 0.0009
Epoch [9/75] - Loss: 0.0009
Epoch [10/75] - Loss: 0.0009
Epoch [11/75] - Loss: 0.0007
Epoch [12/75] - Loss: 0.0012
Epoch [13/75] - Loss: 0.0010
Epoch [14/75] - Loss: 0.0012
Epoch [15/75] - Loss: 0.0011
Epoch [16/75] - Loss: 0.0007
Epoch [17/75] - Loss: 0.0011
Epoch [18/75] - Loss: 0.0011
Epoch [19/75] - Loss: 0.0009
Epoch [20/75] - Loss: 0.0009
Epoch [21/75] - Loss: 0.0007
Epoch [22/75] - Loss: 0.0008
Epoch [23/75] - Loss: 0.0010
Epoch [24/75] - Loss: 0.0011
Epoch [25/75] - Loss: 0.0007
Epoch [26/75] - Loss: 0.0011
Epoch [27/75] - Loss: 0.0010
Epoch [28/75] - Loss: 0.0011
Epoch [29/75] - Loss: 0.0008
Epoch [30/75] - Loss: 0.0011
Epoch [31/75] - Loss: 0.0010
Epoch [32/75] - Loss: 0.0011
Epoch [33/75] - Loss: 0.0009
Epoch [34/75] - Loss: 0.0013
Epoch [35/75] - Loss: 0

### Gaussian Blur Pipeline

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class GaussianBlur(nn.Module):
    def __init__(self, kernel_size=5, sigma=1.0):
        super().__init__()
        self.kernel_size = kernel_size
        self.sigma = sigma
        self.padding = kernel_size // 2
        self.kernel = self.create_gaussian_kernel()

    def create_gaussian_kernel(self):
        k = self.kernel_size
        sigma = self.sigma

        # Create 1D Gaussian kernel
        x = torch.arange(-k // 2 + 1., k // 2 + 1.)
        gauss = torch.exp(-x**2 / (2 * sigma**2))
        gauss = gauss / gauss.sum()

        # Create 2D Gaussian kernel by outer product
        kernel_2d = torch.outer(gauss, gauss)
        kernel_2d = kernel_2d.expand(3, 1, k, k)  # [C_out, C_in/groups, H, W]
        return kernel_2d

    def forward(self, x):
        kernel = self.kernel.to(x.device)
        return F.conv2d(x, kernel, padding=self.padding, groups=3)

In [27]:
class BlurCameraPipeline(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.blur = GaussianBlur(kernel_size=5, sigma=1.0)
        self.cnn = SimpleCNN(num_classes)

    def forward(self, x):
        x = self.blur(x)
        return self.cnn(x)

In [30]:
model = BlurCameraPipeline(num_classes=10).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

for epoch in range(75):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/75] - Loss: {running_loss / len(train_loader):.4f}")


Epoch [1/75] - Loss: 2.0624
Epoch [2/75] - Loss: 1.9013
Epoch [3/75] - Loss: 1.8339
Epoch [4/75] - Loss: 1.7905
Epoch [5/75] - Loss: 1.7480
Epoch [6/75] - Loss: 1.7115
Epoch [7/75] - Loss: 1.6839
Epoch [8/75] - Loss: 1.6636
Epoch [9/75] - Loss: 1.6441
Epoch [10/75] - Loss: 1.6282
Epoch [11/75] - Loss: 1.6130
Epoch [12/75] - Loss: 1.6032
Epoch [13/75] - Loss: 1.5916
Epoch [14/75] - Loss: 1.5761
Epoch [15/75] - Loss: 1.5664
Epoch [16/75] - Loss: 1.5557
Epoch [17/75] - Loss: 1.5441
Epoch [18/75] - Loss: 1.5334
Epoch [19/75] - Loss: 1.5268
Epoch [20/75] - Loss: 1.5190
Epoch [21/75] - Loss: 1.5107
Epoch [22/75] - Loss: 1.5038
Epoch [23/75] - Loss: 1.4980
Epoch [24/75] - Loss: 1.4879
Epoch [25/75] - Loss: 1.4822
Epoch [26/75] - Loss: 1.4765
Epoch [27/75] - Loss: 1.4703
Epoch [28/75] - Loss: 1.4656
Epoch [29/75] - Loss: 1.4572
Epoch [30/75] - Loss: 1.4509
Epoch [31/75] - Loss: 1.4452
Epoch [32/75] - Loss: 1.4386
Epoch [33/75] - Loss: 1.4319
Epoch [34/75] - Loss: 1.4265
Epoch [35/75] - Loss: 1

In [31]:
# -------- Evaluation --------
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

acc = correct / total
print(f"Test Accuracy: {acc * 100:.2f}%")

# -------- Save Model --------
torch.save(model.state_dict(), "differentiable_gaussian_blur_pipeline.pth")
print("Model saved to gaussian_blur_pipeline.pth")

Test Accuracy: 54.26%
Model saved to gaussian_blur_pipeline.pth
